# Assignment

In this assignment we want to get comfortable with loading and manipulating data in Python. While future assignments will focus more using structured data which we can load into a `DataFrame` using `pandas`, this assignment is focused on semi-structured data and how we can "flatten" it and then load it into other formats. The objective is to see how data flows in Python from one object to another and what advantages and disadvantages each offers.

Let's read the `books.json` data set and display the first item in it.

In [1]:
import json
with open('../data/books.json', encoding = 'utf-8') as f:
    books_dict = json.load(f)

from pprint import pprint
pprint(books_dict[0]) # print information for the first book

{'_id': 1,
 'authors': ['W. Frank Ableson', 'Charlie Collins', 'Robi Sen'],
 'categories': ['Open Source', 'Mobile'],
 'isbn': '1933988673',
 'longDescription': 'Android is an open source mobile phone platform based on '
                    'the Linux operating system and developed by the Open '
                    'Handset Alliance, a consortium of over 30 hardware, '
                    'software and telecom companies that focus on open '
                    'standards for mobile devices. Led by search giant, '
                    'Google, Android is designed to deliver a better and more '
                    'open and cost effective mobile experience.    Unlocking '
                    "Android: A Developer's Guide provides concise, hands-on "
                    'instruction for the Android operating system and '
                    'development tools. This book teaches important '
                    'architectural concepts in a straightforward writing style '
                    

1. Write a program that goes through the entire data and extracts the following information:  <span style="color:red" float:right>[4 point]</span>

  - title of the book
  - name of the first author
  - name of the second author (if book has more than one author)
  - number of authors
  - ISBN
  - if the word "data" is in the book's description
  - the number of words in the book's description
  - the year the book was published

  Of course because JSON data doesn't necessarily enforce any sort of schema, we can't be sure that the information we are trying to extract exists for every book. For example, if the book only has one author, then there is no second author. So use `try` and `except` as you loop through every book and skip to the next item every time some information is missing.

  Store the extracted data in a list named `rows` whose elements are tuples, one tuple per book. For example, the first element of `rows` stores the tuple for the first book and should look like this:

        ('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', True, 252, 2009)

I did this first to practice. I wanted to replicate the output above and call the first book. This taught me many things about the data. Including that the format of the date needed to be changed to match what is above. Also I practiced using len()

In [2]:
title = books_dict[0]["title"]
author_1 = books_dict[0]["authors"][0]
author_2 = books_dict[0]["authors"][1]
num_authors = len(books_dict[0]['authors'])
isbn = books_dict[0]["isbn"]
has_data = "data" in books_dict[0]["longDescription"]
desc_len = len(books_dict[0]["longDescription"].split())
year_published = int(books_dict[0]["publishedDate"]["$date"][0:4]) # this is the change the format of the date

row_1 = (title, author_1, author_2, num_authors, isbn, has_data, desc_len, year_published)
print(row_1)

('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', True, 252, 2009)


I used a For loop to create one tuple per book. I used the try and except function inorder to skip over any missing data 
in a particular variable. 
However by using try and except I learned that this can mask any errors that might have occured otherwise. 

I realized for the questions having to do with the long description you need to use the .split function.
For the question about the number of words (desc_len) you need the .split function in order to count only the words and not empty space ( in addition to the len function). There were multiple description fields but I made the assumption that we were meant to use long description to created the desc_len variable.

For the has_data variable you need the split function so you dont count words like dataset. In addition, I used "data" in to determine if the word data appeared anywhere. 

Instead of using .split for both the desc_len variable and the has_data variable I challenged myself to see if I could create a new variable to plug into these two. I created longDescriptionWords which using the split function to create a list of words with no spaces. This way I can reuse code and be more efficient. 

For the year variable I needed to change the formatting so I used int along wiht $date and also 0:4
to indicate that I wanted the first 4 digits. 
I created the row variable to added each book to the list of rows. 



In [3]:


rows = []

for book in books_dict: # The use of the "for" loop creates a variable that changes its values each time. 
                        # In the first loop its the first book, the second time its the second book etc.
    longDescriptionWords = [] #this needs to be here incase there isn't a long description 
    
    try:
        longDescriptionWords = book["longDescription"].split() 
        desc_len = len(longDescriptionWords)
    except:
        desc_len = None
    
    try:
        title = book["title"]
    except:
        title = None
    
    try:
        num_authors = len(book['authors'])
    except:
        num_authors = None
        
    try:
      author_2 = book["authors"][1]
    except:
      author_2 = None
    
    try: 
       author_1 = book["authors"][0]
    except:
        author_1 = None
        
    
    try:
        isbn = book["isbn"]
    except:
        isbn = None
    
    try: 
        has_data = "data" in longDescriptionWords 
    except:
        has_data = None
        
    
    
    try:
        year_published = int(book["publishedDate"]["$date"][0:4])
    except: 
        year_published = None
    
    row= (title, author_1, author_2, num_authors, isbn, has_data, desc_len, year_published)

    rows.append(row) #add to the list of rows




2. Save the content of `rows` in a SQL-like table using `sqlite3`, and choose the appropriate column types. <span style="color:red" float:right>[2 point]</span> 

  As your column names use the following:

  - `title`
  - `author_1`
  - `author_2`
  - `num_authors`
  - `isbn`
  - `has_data`
  - `desc_len`
  - `year_published`

To save the content of rows in a SQL-like table I imported sqlite3 made a connection and a cursor. In the cursor.execute line 
I created a table with column names along with the data type. For example text when it was a string and int when it was an integer. 
For cursor.executemany runs the code for every item in list replacing the question marks with each value in order. 

In [4]:
import sqlite3

connection = sqlite3.connect(':memory:') 
cursor = connection.cursor()

cursor.execute('''CREATE TABLE books_long
             (title text, author_1 text, author_2 text, num_authors int, isbn text, has_data BOOL, desc_len text, 
             year_published int)''')


cursor.executemany('INSERT INTO books_long VALUES (?,?,?,?,?,?,?,?)', rows) # You need as many question marks as you have columns

connection.commit() # save the changes



3. Write a SQL query against the table to show all books that (1) contain the word "data" and (2) have more than 3 authors. Store the result of the query in an object called `books_table`, then close the connection. <span style="color:red" float:right>[2 point]</span>

The cursor.execute is broken up in two sections: the select section and the filter section. 
Select * means select all columns from the data. The where clause is used to filter the data. 

I used the previously created variable has_data to determine if the book contain the word data. I made the assumption that this 
is what was meant however I guess that I could have also used title to see if data was in the book title or I could have also
used the short description. 

connection.close closes the connection.


In [5]:
book_table =(cursor.execute('''select * from books_long where has_data is True and num_authors > 3''').fetchall())

print(len(book_table)) # this was to check how many books met the requirements of containing the word "data" and 
                       # having more than three authors
connection.close

13


<function Connection.close>

SQL tables are not the only way, and definitely not the most straightforward way to store and manipulate data in Python. A format that's more popular with data scientist is to use the `pandas` library to create a `DataFrame`. This library has a lot of functionality that makes it easy to run the common tasks data scientists do with data.

4. Read the data from the above query into a `DataFrame` and call it `books_df`. HINT: Use `pd.DataFrame` and specify meaningful column names to use for the columns. <span style="color:red" float:right>[1 point]</span>

I used pd.DataFrame to create a data frame in pandas. You need to specify the data and the column names. When I printed the 
data frame I was confused that the dates were stored as floats. I looked this up and found out that NaN can't be stored in an integer array, 
so thats why its displayed as a float.
I orginially used the data book_table because thats what I thought "the above query meant" however on Piazza the question on which data to used was asked and the answer JSON data was given so I changed it to rows instead. 

In [6]:
import pandas as pd
book_df = pd.DataFrame.from_records(data = rows, columns = ["title", "author_1", "author_2", "num_authors","isbn", "has_data", "desc_len", "year_published"]) 

pprint(book_df) 

                                         title                      author_1  \
0                            Unlocking Android              W. Frank Ableson   
1            Android in Action, Second Edition              W. Frank Ableson   
2                     Specification by Example                   Gojko Adzic   
3                             Flex 3 in Action  Tariq Ahmed with Jon Hirschi   
4                             Flex 4 in Action                   Tariq Ahmed   
..                                         ...                           ...   
426                             DSLs in Action                          None   
427  Database Programming for Handheld Devices                          None   
428           Jakarta Commons Online Bookshelf                          None   
429                   Browsing with HttpClient                          None   
430               Codec: Encoders and Decoders                          None   

            author_2  num_authors      

5. Display the first few columns of a `DataFrame` by calling its `head` method. <span style="color:red" float:right>[1 point]</span>

Using .head will print the first few rows of the data frame. This is very useful in being able to look at the data and make sure
everything looks like you are expecting it to. Something I learned that even the table will start counting at 0 which I wasn't
expecting. 

In [7]:
book_df.head()

,title,author_1,author_2,num_authors,isbn,has_data,desc_len,year_published
0,Unlocking Android,W. Frank Ableson,Charlie Collins,3,1933988673,True,252.0,2009.0
1,"Android in Action, Second Edition",W. Frank Ableson,Robi Sen,2,1935182722,False,101.0,2011.0
2,Specification by Example,Gojko Adzic,None,1,1617290084,False,NaN,2011.0
3,Flex 3 in Action,Tariq Ahmed with Jon Hirschi,Faisal Abid,2,1933988746,True,254.0,2009.0
4,Flex 4 in Action,Tariq Ahmed,Dan Orlando,4,1935182420,True,329.0,2010.0


Remember how earlier we said that a `DataFrame` is built on top of `numpy` arrays? Another way of saying it is that a `DataFrame` is an **abstraction** on top of `numpy` arrays: i.e. a `DataFrame` is a more **high-level** object than a `numpy` array. 

6. Call the `values` attribute of your `DataFrame` to convert it into a numpy array and display the first 3 elements of the array. <span style="color:red" float:right>[1 point]</span>

I used .values to call the first three elements. It is important to remember to start at 0 and include 3. 

In [8]:
print(book_df.values[0:3])

[['Unlocking Android' 'W. Frank Ableson' 'Charlie Collins' 3 '1933988673'
  True 252.0 2009.0]
 ['Android in Action, Second Edition' 'W. Frank Ableson' 'Robi Sen' 2
  '1935182722' False 101.0 2011.0]
 ['Specification by Example' 'Gojko Adzic' None 1 '1617290084' False nan
  2011.0]]


Now you can judge which object is more "user-friendly". That's one of the things that abstractions allow us to do: build more user-friendly (abstract) objects from less user-friendly (but more fundamental) objects.

# End of assignment